<a href="https://colab.research.google.com/github/dhruv0000/neural-robot-dynamics/blob/main/train_colab.ipynb%20latest%20run" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Robot Dynamics Training on Colab

This notebook demonstrates how to setup the environment, generate a dataset, and train the NeRD model.

In [1]:
# 1. Setup Environment
!git clone https://github.com/dhruv0000/neural-robot-dynamics.git
%cd neural-robot-dynamics
!pip install -r requirements.txt
!pip install warp-lang==1.8.0
!pip install rl_games

Cloning into 'neural-robot-dynamics'...
remote: Enumerating objects: 444, done.
remote: Counting objects: 100% (444/444), done.
remote: Compressing objects: 100% (326/326), done.
remote: Total 444 (delta 101), reused 416 (delta 74), pack-reused 0 (from 0)
Receiving objects: 100% (444/444), 17.28 MiB | 18.68 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Filtering content: 100% (11/11), 202.03 MiB | 71.46 MiB/s, done.
/content/neural-robot-dynamics
ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Ignored the following versions that require a different python version: 1.10.0 Requires-Python <3.12,>=3.8; 1.10.0rc1 Requires-Python <3.12,>=3.8; 1.10.0rc2 Requires-Python <3.12,>=3.8; 1.10.1 Requires-Python <3.12,>=3.8; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.

In [2]:
# 2. Generate Dataset
# We generate a smaller dataset for demonstration purposes.

%cd generate

# Generate Training Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 10000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_train.hdf5 --trajectory-length 100 --num-envs 64 --seed 0

# Generate Validation Data
!python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 2000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_valid.hdf5 --trajectory-length 100 --num-envs 64 --seed 10

%cd ..

/content/neural-robot-dynamics/generate
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
 [NeuralEnvironment] Creating abstract contact environment: Cartpole. 
Creating 64 environments: 100% 64/64 [00:00<00:00, 188.84it/s]
Module warp.sim.integrator_featherstone 18b3327 load on device 'cuda:0' took 21206.10 ms  (compiled)
Module envs.abstract_contact_environment 8e8d790 load on device 'cuda:0' took 575.82 ms  (compiled)
Module integrators.integrator_neural ee402cd load on device 'cuda:0' took 749.49 ms  (compiled)
 [NeuralEnvironment] Created a DUMMY Neural Integrator. 
  0% 0/10000 [00:00<?, ?it/s]Module utils.warp_utils 294c46a load on device 'cuda:0' took 668.76 ms  (compile

In [3]:
# 3. Train Baseline Model (Transformer)
%cd train

import yaml
import os

# Load default config
with open('cfg/Cartpole/transformer.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Override dataset paths to point to the generated data
cfg['algorithm']['dataset']['train_dataset_path'] = '../data/datasets/Cartpole/trajectory_len-100_train.hdf5'
cfg['algorithm']['dataset']['valid_datasets']['exp_trajectory'] = '../data/datasets/Cartpole/trajectory_len-100_valid.hdf5'

# Reduce training parameters for quick demonstration
cfg['algorithm']['num_epochs'] = 5
cfg['algorithm']['num_iters_per_epoch'] = 100

# Save the modified config
with open('colab_config.yaml', 'w') as f:
    yaml.dump(cfg, f)

# Run training
!python train.py --cfg colab_config.yaml --logdir ../data/logs/baseline

/content/neural-robot-dynamics/train
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 17:57:10.041161: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763920630.060345    1294 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763920630.066117    1294 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763920630.080930    1294

In [4]:
# 4. Train Mamba Model
# We use the same config but add the --novelty mamba flag
!python train.py --cfg colab_config.yaml --novelty mamba --logdir ../data/logs/mamba

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 17:58:26.902536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763920706.934916    2077 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763920706.944724    2077 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763920706.968518    2077 computation_placer.cc:177] computati

In [5]:
# 5. Train Unroll Model
# We use the same config but add the --novelty unroll flag
!python train.py --cfg colab_config.yaml --novelty unroll --logdir ../data/logs/unroll

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-23 18:02:18.407934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763920938.427338    3493 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763920938.433641    3493 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763920938.448743    3493 computation_placer.cc:177] computati

In [10]:
# 6. RL Evaluation
# Evaluate the trained models using the pretrained RL policy
import os
import glob
import subprocess

def find_latest_model(model_type):
    base_log_dir = f'../data/logs/{model_type}'
    if not os.path.exists(base_log_dir):
        print(f'Log dir not found: {base_log_dir}')
        return None
    dirs = [d for d in glob.glob(os.path.join(base_log_dir, '*')) if os.path.isdir(d)]
    if not dirs:
        print(f'No logs found for {model_type}')
        return None
    latest_dir = sorted(dirs)[-1]
    model_path = os.path.join(latest_dir, 'nn', 'best_eval_model.pt')
    if not os.path.exists(model_path):
        print(f'Model file not found: {model_path}')
        return None
    return model_path

models = ['baseline', 'mamba', 'unroll']
for model in models:
    print(f'Evaluating {model.capitalize()} Model...')
    model_path = find_latest_model(model)
    if model_path:
        print(f'Using model: {model_path}')

        # Convert paths to absolute to avoid issues with subprocess cwd
        abs_model_path = os.path.abspath(model_path)
        abs_playback_path = os.path.abspath('../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth')
        abs_rl_cfg_path = os.path.abspath('../eval/eval_rl/cfg/Cartpole/cartpole.yaml')

        cmd = [
            'python', 'run_rl.py',
            '--rl-cfg', abs_rl_cfg_path,
            '--playback', abs_playback_path,
            '--num-envs', '1',
            '--num-games', '5',
            '--env-mode', 'neural',
            '--nerd-model-path', abs_model_path
        ]

        try:
            subprocess.run(cmd, cwd='../eval/eval_rl', check=True, capture_output=True, text=True)
        except subprocess.CalledProcessError as e:
            print(f'Error running RL evaluation for {model}:')
            print('STDOUT:', e.stdout)
            print('STDERR:', e.stderr)
            raise e
    else:
        print(f'Skipping {model} evaluation.')

Evaluating Baseline Model...
Using model: ../data/logs/baseline/11-23-2025-17-57-17/nn/best_eval_model.pt
Evaluating Mamba Model...
Using model: ../data/logs/mamba/11-23-2025-17-58-33/nn/best_eval_model.pt
Evaluating Unroll Model...
Using model: ../data/logs/unroll/11-23-2025-18-02-25/nn/best_eval_model.pt
